In [1]:
import os
import numpy as np
import pandas as pd
import csv

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Embedding, Dropout, Bidirectional

from sklearn.preprocessing import LabelBinarizer

# The next imports are only needed for the preprocessing
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from utils.nlputil import preprocess_text

ImportError: No module named 'keras'

In [2]:
tweet_tokenizer = TweetTokenizer()
wordnet_lemmatizer = WordNetLemmatizer()

NameError: name 'TweetTokenizer' is not defined

In [3]:
NUM_LABELS = 3
MAX_WORDS = 1000
EMBEDDING_DIM = 50
LSTM_HIDDEN_DIM = 32

In [4]:
df_tweets_train = pd.read_csv('training.csv')
df_tweets_train.head()

,tweet,senti
0,@united UA5396 can wait for me. I'm on the gro...,0
1,I hate Time Warner! Soooo wish I had Vios. Can...,0
2,Tom Shanahan's latest column on SDSU and its N...,2
3,Found the self driving car!! /IWo3QSvdu2,2
4,@united arrived in YYZ to take our flight to T...,0


### Preprocess training and test data

In [19]:
train_tweets = df_tweets_train['tweet']
train_polarities = df_tweets_train['senti']

train_tweets_processed = [''] * len(train_tweets)

for idx, doc in enumerate(train_tweets):
    train_tweets_processed[idx] = preprocess_text(doc, tokenizer=tweet_tokenizer, lemmatizer=wordnet_lemmatizer)

In [5]:
df_tweets_test = pd.read_csv('test.csv')

test_tweets = df_tweets_test['tweet']
test_polarities = df_tweets_test['senti']  

test_tweets_processed = [''] * len(test_tweets)

for idx, doc in enumerate(test_tweets):
    test_tweets_processed[idx] = preprocess_text(doc, tokenizer=tweet_tokenizer, lemmatizer=wordnet_lemmatizer)  

FileNotFoundError: File b'test.csv' does not exist

### Prepare labels

In [21]:
encoder = LabelBinarizer()
encoder.fit(train_polarities)
y_train = encoder.transform(train_polarities)
y_test = encoder.transform(test_polarities)

print(y_test[:10])

[[1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 0]]


In [22]:
longest_train_tweet = max([len(s.split()) for s in train_tweets_processed])
longest_test_tweet = max([len(s.split()) for s in test_tweets_processed])

max_seq_len = max(longest_train_tweet, longest_test_tweet)

print("Maximum sequence length: {}".format(max_seq_len))

Maximum sequence length: 29


In [23]:
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(train_tweets_processed)

In [24]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 2718 unique tokens.


In [25]:
X_train = tokenizer.texts_to_sequences(train_tweets_processed)
X_test = tokenizer.texts_to_sequences(test_tweets_processed)

max_idx = max([ max(l) for l in X_train if len(l) > 0])

print(X_train[0])
print("Largest used index: {}".format(max_idx)) # This should be (MAX_WORDS-1)


[4, 846, 52, 8, 506, 178, 6, 204, 344, 507, 508]
Largest used index: 999


In [26]:
X_train = pad_sequences(X_train, maxlen=max_seq_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_seq_len, padding='post', truncating='post')

print(X_train[0])
print("Sequence length: {}".format(len(X_train[0])))

[  4 846  52   8 506 178   6 204 344 507 508   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0]
Sequence length: 29


## Training the model

### Load pretrained word embeddings (GloVe)

In [6]:
df_glove = pd.read_table('glove.6B.50d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

# Print the first 5 lines
df_glove.head()

FileNotFoundError: File b'glove.6B.50d.txt' does not exist

In [28]:
oov_words = set()

embedding_matrix = np.random.random((MAX_WORDS, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = df_glove.loc[word].as_matrix()
        try:
            embedding_matrix[i] = embedding_vector
        except:
            pass
        #print(">>>>", word)
    except Exception as e:
        oov_words.add(word)
        
        
print("Number of words not in the pretrained set: {}".format(len(oov_words)))
print("Ratio of words not in the pretrained set: {:.2}".format(len(oov_words)/len(word_index)))

Number of words not in the pretrained set: 523
Ratio of words not in the pretrained set: 0.19


In [29]:
model = Sequential()

# Choose one of the following 2 lines
#model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=max_seq_len))
#model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=max_seq_len, weights=[embedding_matrix], trainable=False))
model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=max_seq_len, weights=[embedding_matrix], trainable=True))

model.add(Dropout(0.5)) # Optional

# Choose none or one of the following 2 lines
#model.add(LSTM(LSTM_HIDDEN_DIM, return_sequences=True))
model.add(Bidirectional(LSTM(LSTM_HIDDEN_DIM, return_sequences=True)))

model.add(Dropout(0.5)) # Optional
model.add(LSTM(max_seq_len))
model.add(Dropout(0.5)) # Optional
model.add(Dense(128))
model.add(Activation('relu'))
#model.add(Dense(64)) # Optional
#model.add(Activation('relu')) # Optional
model.add(Dense(NUM_LABELS))
model.add(Activation('softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 29, 50)            50000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 29, 64)            21248     
_________________________________________________________________
dropout_2 (Dropout)          (None, 29, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 29)                10904     
_________________________________________________________________
dropout_3 (Dropout)          (None, 29)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3840      
__________

### Compile model

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train model

In [31]:
history = model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, validation_split=0.1)

Train on 629 samples, validate on 70 samples
Epoch 1/100
629/629 [==============================] - 2s 3ms/step - loss: 1.0909 - acc: 0.3370 - val_loss: 1.0982 - val_acc: 0.3429
Epoch 2/100
629/629 [==============================] - 1s 2ms/step - loss: 1.0868 - acc: 0.3704 - val_loss: 1.0942 - val_acc: 0.3429
Epoch 3/100
629/629 [==============================] - 1s 2ms/step - loss: 1.0800 - acc: 0.3879 - val_loss: 1.0942 - val_acc: 0.3857
Epoch 4/100
629/629 [==============================] - 1s 2ms/step - loss: 1.0809 - acc: 0.3895 - val_loss: 1.0927 - val_acc: 0.3429
Epoch 5/100
629/629 [==============================] - 1s 2ms/step - loss: 1.0813 - acc: 0.3736 - val_loss: 1.0909 - val_acc: 0.3857
Epoch 6/100
629/629 [==============================] - 1s 2ms/step - loss: 1.0783 - acc: 0.3752 - val_loss: 1.0978 - val_acc: 0.3429
Epoch 7/100
629/629 [==============================] - 1s 2ms/step - loss: 1.0803 - acc: 0.3879 - val_loss: 1.0931 - val_acc: 0.3857
Epoch 8/100
629/629 [===

Epoch 62/100
629/629 [==============================] - 1s 2ms/step - loss: 0.5216 - acc: 0.8076 - val_loss: 1.1189 - val_acc: 0.7000
Epoch 63/100
629/629 [==============================] - 1s 2ms/step - loss: 0.4716 - acc: 0.8124 - val_loss: 1.1696 - val_acc: 0.7000
Epoch 64/100
629/629 [==============================] - 1s 2ms/step - loss: 0.4717 - acc: 0.8076 - val_loss: 1.3808 - val_acc: 0.5857
Epoch 65/100
629/629 [==============================] - 1s 2ms/step - loss: 0.4992 - acc: 0.8045 - val_loss: 1.1646 - val_acc: 0.6571
Epoch 66/100
629/629 [==============================] - 1s 2ms/step - loss: 0.4576 - acc: 0.8219 - val_loss: 1.0873 - val_acc: 0.7143
Epoch 67/100
629/629 [==============================] - 1s 2ms/step - loss: 0.4244 - acc: 0.8331 - val_loss: 1.1563 - val_acc: 0.7143
Epoch 68/100
629/629 [==============================] - 1s 2ms/step - loss: 0.4384 - acc: 0.8235 - val_loss: 1.2359 - val_acc: 0.7143
Epoch 69/100
629/629 [==============================] - 1s 2ms

### Evaluate model

In [32]:
score = model.evaluate(X_test, y_test, batch_size=32, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

298/298 [==============================] - 0s 471us/step
Test score: 0.946238966996
Test accuracy: 0.708053692075
